In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import emoji
import csv
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
df=pd.read_csv('../data/raw/raw_data.csv',encoding='latin1')

In [26]:
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [27]:
df.info

<bound method DataFrame.info of         _unit_id  _golden _unit_state  _trusted_judgments _last_judgment_at  \
0      815719226    False   finalized                   3    10/26/15 23:24   
1      815719227    False   finalized                   3    10/26/15 23:30   
2      815719228    False   finalized                   3    10/26/15 23:33   
3      815719229    False   finalized                   3    10/26/15 23:10   
4      815719230    False   finalized                   3     10/27/15 1:15   
...          ...      ...         ...                 ...               ...   
20045  815757572     True      golden                 259               NaN   
20046  815757681     True      golden                 248               NaN   
20047  815757830     True      golden                 264               NaN   
20048  815757921     True      golden                 250               NaN   
20049  815757985     True      golden                 249               NaN   

       gender  gend

## Deleting irrelevant columns

In [28]:
df.drop(['description', '_unit_id','_last_judgment_at','created','fav_number','profileimage','retweet_count','tweet_coord',
         '_trusted_judgments', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone', 
         '_golden','_unit_state', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'sidebar_color', 
         'profile_yn', 'profile_yn:confidence','gender:confidence'], axis=1, inplace=True)

In [29]:
df.head()

,gender,text
0,male,Robbie E Responds To Critics After Win Against...
1,male,ÛÏIt felt like they were my friends and I was...
2,male,i absolutely adore when louis starts the songs...
3,male,Hi @JordanSpieth - Looking at the url - do you...
4,female,Watching Neighbours on Sky+ catching up with t...


## Remove null values from dataset

In [30]:
df.isna().sum()

gender    97
text       0
dtype: int64

In [31]:
df.dropna(axis=0,inplace=True)

## Removing Data that doesn't have correct gender

In [32]:
df['gender'].value_counts()

female     6700
male       6194
brand      5942
unknown    1117
Name: gender, dtype: int64

In [33]:
df = df[(df['gender'] == 'female') | (df['gender'] == 'male')]

In [34]:
df['gender'].value_counts()

female    6700
male      6194
Name: gender, dtype: int64

## Make gender Binary

In [35]:
for gen in df['gender']:
  if gen=='male':
     df['gender'].replace({'male':'1'},inplace=True)
  elif gen=='female':
     df['gender'].replace({'female':'0'},inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5832\3395811361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'].replace({'male':'1'},inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5832\3395811361.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'].replace({'female':'0'},inplace=True)


In [36]:
df['gender'].value_counts()

0    6700
1    6194
Name: gender, dtype: int64

### Clean text Column

In [67]:
def clean(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    emoji.replace_emoji(tweet, "")
    tweet = tweet.encode("ascii", "ignore").decode("ascii")
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    return tweet

In [68]:
df['text'] = pd.DataFrame(df['text'].apply(lambda x: clean(x)))
df.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5832\683280358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = pd.DataFrame(df['text'].apply(lambda x: clean(x)))


,gender,text
0,1,Robbie E Responds To Critics After Win Against...
1,1,It felt like they were my friends and I was li...
2,1,i absolutely adore when louis starts the songs...
3,1,Hi - Looking at the url - do you use ?! Don't ...
4,0,Watching Neighbours on Sky+ catching up with t...


In [69]:
df.columns

Index(['gender', 'text'], dtype='object')

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12894 entries, 0 to 20049
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  12894 non-null  object
 1   text    12894 non-null  object
dtypes: object(2)
memory usage: 302.2+ KB


# Saving clean data

In [71]:
df.to_csv('../data/clean/clean_data.csv', index=False)